Naive Code to Create a list of Ranked suffixes

In [5]:
from pprint import pprint
# def suffix_dict(s):
#     suffixDict = {}
#     for i in range(len(s)):
#         suffixDict[i] = s[i:]
#     return suffixDict

def return_suffixes(s):
    suffixes = []
    for i in range(len(s)):
        suffixes.append(s[i:])
    return list(zip([i+1 for i in range(len(string))], suffixes))

def return_ranked_suffixes(s):
    suffixes = return_suffixes(s)
    suffixes.sort(key = lambda x: x[1])
    rankToSuffixID = list(zip([rank for rank in range(1,len(string)+1)], [suffix for suffix in suffixes]))
    rankToSuffixID.sort(key = lambda x: x[0])
    return rankToSuffixID

string = "mississippi$"
# #string = "abcabxabcd$"
# pprint(suffix_dict(string))
pprint("Rank : Suffix ID : Suffix ")
pprint(return_ranked_suffixes(string))

'Rank : Suffix ID : Suffix '
[(1, (12, '$')),
 (2, (11, 'i$')),
 (3, (8, 'ippi$')),
 (4, (5, 'issippi$')),
 (5, (2, 'ississippi$')),
 (6, (1, 'mississippi$')),
 (7, (10, 'pi$')),
 (8, (9, 'ppi$')),
 (9, (7, 'sippi$')),
 (10, (4, 'sissippi$')),
 (11, (6, 'ssippi$')),
 (12, (3, 'ssissippi$'))]


In [6]:
class Edge:
    def __init__(self, start, string, end = -1):
        self.start_value = string[start]
        self.start = start
        self.end = end

    def value(self, string, global_end):
        if self.end == -1: ## If not a leaf edge
            return string[self.start : global_end + 1]
        else:
            return string[self.start : self.end + 1]
    
    def length(self, global_end):
        if self.end == -1: ## If not a leaf edge
            return global_end - self.start
        else:
            return self.end - self.start
        
    def __hash__(self):
        # Hash the edge object based on the value attribute
        return hash(self.value[0])

    def __eq__(self, other):
        # Define equality based on the value attribute
        return self.start_value == other.start_value if isinstance(other, Edge) else False

    def __repr__(self):
        # This method helps in giving a printable representation of the object
        return f"Edge({self.value})"



In [7]:

class Node:
    def __init__(self, incoming_edge = None, link_to = None):
        self.incoming_edge = incoming_edge
        self.link_to = link_to
        self.children = {} ## HashMap<Edge: Node>

class RootNode(Node):
    def __init__(self, incoming_edge = None, link_to = None):
        self.incoming_edge = incoming_edge
        self.link_to = link_to
        self.children = {} ## HashMap<Edge: Node>

        ## HashMap mapping characters to their Edges
        self.char_to_edge = {} ## HashMap<Char: Edge>

class SuffixTree:
    def __init__(self, string):
        self.root = RootNode()
        self.root.link_to = self.root

        ## Triple representing the active point
        self.aNode = self.root 
        self.aEdge = None ## TODO: Should this hold an edge object later on
        self.aLength = 0 
    
        self.remainder = 0
        self.text = string
        self.end = -1 ## Represents "#"
        self.new_internal_node = None ## Holds the last created internal node
        

    def _edge_insert(self, j):
        '''
        Insert a new node at the active point which lies along an edge
        
        If we are inserting an internal node at position x along edge RP then must make the following changes
        x = active length
        j = position, index of current phase
        R = active node
        P = leaf node

        1. Update R to remove P: {RP:P}
        2. Update RP edge to become RG edge by setting end = x
        3. Create new internal node G, incoming edge = RG
        4. Update R to add G: {RG:G}

        5. Create new edge GO, start = j end = -1
        6. Create new leaf node O, incoming edge = GO
        7. Update G to add O: {GO:O}

        8. Create new edge GP, start = x, end = -1
        9. Update leaf node P, incoming edge = GP

        PREVIOUS:    (R)---x---(P) 

                          (O)
        NEW:               |     
                    (R)---(G)---(P)
        '''
        x = self.aLength
        R = self.aNode
        RP = R.incoming_edge

        P = R.children.pop(RP) ## 1
        RP.end = x ## 2

        G = Node(incoming_edge = RP, link_to=self.root) ## 3
        R.children[RP] = G ## 4

        GO = Edge(start = j, end = -1, string = self.text) ##5
        O = Node(incoming_edge = GO, link_to = self.root) ## 6 ## TODO: Verify these suffix links
        G.children[GO] = O ## 7

        GP = Edge(start = x, end = -1, string = self.text) ## 8
        P.incoming_edge = GP ## 9

        ## Set new_internal_node to G
        self.new_internal_node = G


    def _node_insert(self, position):
        '''
        Insert a new node at the active node
        '''
        new_edge = Edge(start = position, string = self.text)
        new_leaf_node = Node(incoming_edge = new_edge, link_to = self.root)
        self.aNode.children[new_edge] = new_leaf_node

        ## If this node in the root node, add the edge_value : edge into the char_to_edge hashmap for O(1)
        ## lookup when "incrementing" active edge after an intertion at the root node : [start_character] = new_edge
        if self.aNode == self.root:
            self.aNode.char_to_edge[self.text[new_edge.start]] = new_edge


    def _check_or_insert(self, position): ## --> Returns True if an insertion was made, False if it already existed
        '''
        Checks if character exists at current active point, and if it doesn't, then insert it
        position = str[j-k] where j is the index of current phase, and k is obtained from iterating through range(remainder)
        str[j-k] where j is the index of current phase, and k is obtained from iterating through range(remainder)
        '''
        ## If we are on a node
        if self.aLength == 0: 

            ## If the character exists as a child node, it is implicitly represented, so return
            if self.text[position] in self.aNode.children.keys():
                return False
            
            ## If it doesn't exist, add the character as a new edge and node
            else:
                self._node_insert(position)
                return True

        ## Else we are on an edge
        else:
            edge_length = self.aEdge.length(self.end)

            ## If length of the edge <= the remaining length we must traverse (aLength), then recurse deeper into tree
            if self.aLength >= edge_length:
                self.aNode = self.aNode.children[str[position - edge_length]] ## aNode points to node traversed to 
                self.aEdge = self.aNode.incoming ## aEdge points to edge just traversed
                self.aLength -= edge_length ## Decrement aLength to be remaining length 
                return self._check_or_insert(position)
            
            ## Else we don't need to recurse deeper, so check/insert on the current edge
            else:
                edge_value = self.aEdge.value(self.end)

                ## Traverse along the edge for the remaining active length, to check the correct position
                ## If the remainder suffix has already been inserted, return False
                if edge_value[self.aLength] == self.text[position]: ## TODO: Potential off by one error
                    return False
                
                ## Else insert it as a new internal node
                else:
                    self._edge_insert(position) ## TODO: Potential off by one error
                    return True

    def _suffix_link(self, from_node, to_node):
        from_node.link_to = to_node

    def extend_suffix_tree(self, j):
        '''
        Performs a single phase of the suffix tree given. Extends the suffix tree from s[0...j-1] to s[0...j]
        '''

        ## Increment `remainder` for each new character
        self.remainder += 1
        ## Increment end denoter to execute rapid lead expansion
        self.end += 1 
        
        ## Holds the last internal node inserted
        unresolved_internal_node = None
        # Deal with all remaining suffixes. Using for loop to prevent infinite looping
        while self.remainder > 0:
            
            ## Attempt to insert the remaining suffix at the active point `abx`
            ## Doing this traversal (to point of potential insertion) will also update active point to point to where this 
            ## insertion occurred, or where was checked up to 
            inserted = self._check_or_insert(j)

            ## If an insertion did not occur, then str[j] exists at the active point so skip to next phase
            if not inserted:
                
                ## If there exists an unresolved internal node, then link it to the deepest traversed node encountered when checking for remainder
                if unresolved_internal_node is not None:
                    self._suffix_link(from_node = unresolved_internal_node, to_node = self.aNode)

                return 
            
            ## If an insertion did occur
            else: 
                ## Decrement remainder
                self.remainder -= 1

                ## If we are not at the root node, then follow the suffix link from the active node, don't change active_length and active_edge
                if self.aNode != self.root:
                    ## The suffix link will either link to another internal node, or by default, the root note if no alternative suffix link was set
                    self.aNode = self.aNode.link_to  
                    
                ## Else stay at the root node, and decrement active length and increment active edge
                else:
                    self.aLength -= 1

                    ## Increment active_edge to point to the outgoing edge from the root node which corresponds to next char in text
                    next_char_index = j - self.remainder ## abc[abx] j = 6, remainder = 2 ## TODO: Potential off-by-one 
                    next_char = self.text[next_char_index]
                    self.aEdge = self.root.char_to_edge[next_char] ## Retrieve the Edge corresponding to the next Char
                    self.a += 1 ## Increment active_edge. This involve

                ## Resolve suffix link for previously created internal node
                if unresolved_internal_node is not None:
                     self._suffix_link(from_node = unresolved_internal_node, to_node = self.new_internal_node) 

                ## Update unresolved_internal_node
                unresolved_internal_node = new_internal_node  # type: ignore

        ## Reset unresolved_internal_node to None for next phase
        self.new_internal_node = None
    
    def build_suffix_tree(self):
        for j in range(len(self.text)):
            self.extend_suffix_tree(j)

    def print_tree(self, node=None, indent=""):
        if node is None:
            node = self.root
        for key, (start, end, next_node) in node.edges.items():
            text = self.text[start:min(end + 1, self.end + 1)]
            print(f"{indent}{key}: {text} ({start}, {end})")
            self.print_tree(next_node, indent + "  ")

# Example usage
s = "abcabxabcd"
stree = SuffixTree(s)
stree.build_suffix_tree()
stree.print_tree()


TypeError: 'method' object is not subscriptable

In [ ]:
class TimeMap:
    '''
    Data Structure Properties:
    - Stores key-value pairs
    - Efficient search based on key: timestamp (int)
    - Duplicate values possible

    Multiple timestamps are possible for a given key value pair
    HashMap<<Key, Value>, Array<TimeStamps>>

    Constraints: 
    1. Time stamps and Keys are not unique
    2. Entries are set in chronological order, so time stamps will be appended in sorted order

    Assumptions:
    1. Two entries with same key, value and timestamp cannot exist
    '''

    def __init__(self):
        self.entries = {}

    def set(self, key: str, value: str, timestamp: int) -> None:
        if key in self.entries:
            self.entries[key].append([timestamp, value])
        else:
            self.entries[key] = ([[timestamp, value]])

    def get(self, key: str, timestamp: int) -> str:
        if key not in self.entries:
            return ""
        if len(self.entries[key]) == 1:
            return self.entries[key][0][1]
        else: # > 1
            timePairs = self.entries[key]
            left, right = 0, len(timePairs) - 1
            while left < right:
                mid = left + (right - left)//2 ## Prevent stack overflow
                if timePairs[mid][0] == timestamp: ## If timestamp_prev == timestap
                    return timePairs[mid][1]
                elif timePairs[mid][0] < timestamp:
                    left = mid+1
                else: # timePairs[mid][0] > right:
                    right = mid
            return timePairs[left][1] # mid when L == R will hold the value left of timestamp 
    

'''
String get(String key, int timestamp) Returns a value such that set was called previously, with timestamp_prev <= timestamp. 
If there are multiple such values, it returns the value associated with the largest timestamp_prev. 
If there are no values, it returns "".
'''

'\nString get(String key, int timestamp) Returns a value such that set was called previously, with timestamp_prev <= timestamp. \nIf there are multiple such values, it returns the value associated with the largest timestamp_prev. \nIf there are no values, it returns "".\n'

In [ ]:
# Test commands and their corresponding parameters
commands = ["TimeMap", "set", "get", "get", "set", "get", "get"]
params = [[], ["foo", "bar", 1], ["foo", 1], ["foo", 3], ["foo", "bar2", 4], ["foo", 4], ["foo", 5]]
expected_results = [None, None, "bar", "bar", None, "bar2", "bar2"]

# Create an instance of TimeMap
obj = TimeMap()  # the constructor does not take any parameters
results = []

# Iterate over the commands and execute them
for command, param, expected in zip(commands, params, expected_results):
    if command == "TimeMap":
        obj = TimeMap()  # Reinstantiate object
        results.append(None)
    elif command == "set":
        results.append(obj.set(*param))
    elif command == "get":
        result = obj.get(*param)
        results.append(result)
        # Assert to check if the current result matches the expected result
        assert result == expected, f"Error: Expected {expected}, got {result}"

# Optionally, print results and expected results to compare manually (for debugging)
print("Results:", results)
print("Expected:", expected_results)
print("All tests passed!")

Results: [None, None, 'bar', 'bar', None, 'bar2', 'bar2']
Expected: [None, None, 'bar', 'bar', None, 'bar2', 'bar2']
All tests passed!


In [ ]:
from typing import List
from collections import defaultdict

def isNStraightHand(hand: List[int], groupSize: int) -> bool:
    if len(hand)%groupSize > 0:
        return False
    
    cardCount = defaultdict(lambda: 0)

    for num in hand: 
        cardCount[num] += 1

    sorted_cards = sorted(hand)
    for card in sorted_cards:
        print(cardCount)
        num = card
        if cardCount[card] > 0:
            for i in range(groupSize):
                print(num)
                if num not in cardCount:
                    return False
                cardCount[num] -= 1
                if cardCount[num] == 0:
                    cardCount.pop(num)
                num +=1
              
    return True

print(isNStraightHand(hand = [5,1,0,6,4,5,3,0,8,9], groupSize = 2))

defaultdict(<function isNStraightHand.<locals>.<lambda> at 0x7fc240aca040>, {5: 2, 1: 1, 0: 2, 6: 1, 4: 1, 3: 1, 8: 1, 9: 1})
0
1
defaultdict(<function isNStraightHand.<locals>.<lambda> at 0x7fc240aca040>, {5: 2, 0: 1, 6: 1, 4: 1, 3: 1, 8: 1, 9: 1})
0
1
False


In [ ]:
from typing import List
from collections import defaultdict
import heapq

def isNStraightHand(hand: List[int], groupSize: int) -> bool:
    groups = defaultdict(lambda: [])

    heapq.heapify(hand)

    while len(hand) > 0:
        ## Pop out an element
        card = heapq.heappop(hand)
        


    while len(cards) > 0:
        # Update smallest to point to next available smallest card
        smallest = heapq.heappop(hand)
        num = smallest
        print(f"Num:{num}")
        for i in range(groupSize):
            
            if num not in cards:
                return False            

            cards[num] -= 1
            if cards[num] == 0:
                cards.pop(num)
                print(cards)
                print(f"smallest {smallest}")
            num+=1

        print(cards)

    return True

print(isNStraightHand(hand = [1,2,3,6,2,3,4,7,8], groupSize = 3))

In [ ]:
import heapq
d = defaultdict(lambda:[])
print(d)
print(len(d))
s = [1,2,3,6,2,3,4,7,8]
heapq.heapify(s)
print(s)
heapq.heappop(s)
print(s)

defaultdict(<function <lambda> at 0x7fc240a7ab80>, {})
0
[1, 2, 3, 6, 2, 3, 4, 7, 8]
[2, 2, 3, 6, 8, 3, 4, 7]


In [10]:
from collections import deque
from typing import List
def solve(board: List[List[str]]) -> None:
    """
    Form a graph with edges 
    direction vectors
    do a dfs. if any node has children (in each direction vecotr) (direct adjacent)

    either on a border
    or with a node on a border
    so allow the border-ness to emanate from a given o node to all other nodes

    do a BFS from each node in the perimeter if that node is an o node
    mark in place visited nodes with a "v"

    after complete search, go trhough and change all nodes to "x" 
    if not marked with "v", and all nodes marked with "v" to "o"


    invert this logic such that we are doing the bfs from internal nodes??
    """
    ## Direction Vectors
    directions = [(1,0),(-1,0),(0,1),(0,-1)]

    n = len(board)
    q = deque()
    q.append((0,0))
    q.append((n-1,n-1))
    q.append((0,n-1))
    q.append((n-1, 0))

    for i in [0,(n-1)]:
        for j in range(1,n-1):
            q.append((i,j))
            q.append((j,i))

    print(q)
    i = 0
    while q:
        if i > 20:
         break
        i+=1
        ux, uy = q.popleft()
        for dx, dy in directions:
            nx, ny = ux + dx, uy + dy
            #print(nx,ny)
            if 0<nx<n and 0<ny<n and board[nx][ny] == "O": ## If adjacent to a O, then expand to it
                print(True)
                print(nx,ny)
                board[nx][ny] == "V"
                q.append((nx,ny))
                print(q)

    for i in range(n):
        for j in range(n):
            if board[i][j] == "O":
                board[i][j] == "X"

    for i in range(n):
        for j in range(n):
            if board[i][j] == "V":
                board[i][j] == "O"

    return board

board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]
print(solve(board))

deque([(0, 0), (3, 3), (0, 3), (3, 0), (0, 1), (1, 0), (0, 2), (2, 0), (3, 1), (1, 3), (3, 2), (2, 3)])
True
3 1
deque([(0, 1), (1, 0), (0, 2), (2, 0), (3, 1), (1, 3), (3, 2), (2, 3), (3, 1)])
True
1 1
deque([(1, 0), (0, 2), (2, 0), (3, 1), (1, 3), (3, 2), (2, 3), (3, 1), (1, 1)])
True
1 1
deque([(0, 2), (2, 0), (3, 1), (1, 3), (3, 2), (2, 3), (3, 1), (1, 1), (1, 1)])
True
1 2
deque([(2, 0), (3, 1), (1, 3), (3, 2), (2, 3), (3, 1), (1, 1), (1, 1), (1, 2)])
True
1 2
deque([(3, 2), (2, 3), (3, 1), (1, 1), (1, 1), (1, 2), (1, 2)])
True
2 2
deque([(2, 3), (3, 1), (1, 1), (1, 1), (1, 2), (1, 2), (2, 2)])
True
3 1
deque([(2, 3), (3, 1), (1, 1), (1, 1), (1, 2), (1, 2), (2, 2), (3, 1)])
True
2 2
deque([(3, 1), (1, 1), (1, 1), (1, 2), (1, 2), (2, 2), (3, 1), (2, 2)])
True
1 2
deque([(1, 1), (1, 2), (1, 2), (2, 2), (3, 1), (2, 2), (1, 2)])
True
1 2
deque([(1, 2), (1, 2), (2, 2), (3, 1), (2, 2), (1, 2), (1, 2)])
True
2 2
deque([(1, 2), (2, 2), (3, 1), (2, 2), (1, 2), (1, 2), (2, 2)])
True
1 1
dequ

In [8]:
s = "oroqtuunvbeoqpjweoqpdxqpxkos$"
l1 = [8, 12, 7, 11, 13, 15, 14, 5, 16, 1, 3, 6, 10, 9, 4, 17, 2]
l2 = [12, 21, 11, 20, 23, 25, 24, 8, 26, 2, 5, 10, 18, 15, 6, 27, 4, 9, 17, 14, 3, 29, 19, 16, 28, 7, 13, 22, 1]
print(max(l1))
print(sorted(l1))
print(sorted(l2))
print(s)
print(s[0:17-1]+"["+s[17-1]+"]"+s[17:])


17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
oroqtuunvbeoqpjweoqpdxqpxkos$
oroqtuunvbeoqpjw[e]oqpdxqpxkos$
